In [1]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

stock_symbol = "RELIANCE.NS"

from datetime import datetime, timedelta

end_date = datetime.now()
start_date = end_date - timedelta(days=50)  # 2 months (roughly 60 days)

# Convert to strings in the format yfinance expects
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

data = yf.download(stock_symbol, start=start_date_str, end=end_date_str, interval="30m")
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]

print(data.head(1))


[*********************100%***********************]  1 of 1 completed

Price                      Open         High          Low        Close  \
Ticker              RELIANCE.NS  RELIANCE.NS  RELIANCE.NS  RELIANCE.NS   
Datetime                                                                 
2024-12-16 09:30:00      1278.5  1279.800049  1276.150024  1276.300049   

Price                    Volume  
Ticker              RELIANCE.NS  
Datetime                         
2024-12-16 09:30:00      786275  


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [3]:
from sklearn.preprocessing import MinMaxScaler
class dataset(Dataset):
    def __init__(self, data, sequence_length = 15):
        self.data = data

        #Scaling data to range (0,1)
        self.scaler = MinMaxScaler(feature_range=(0,1))
        self.scaled_data = self.scaler.fit_transform(data)

        self.images = []
        self.output = []
        for i in range (len(data) - sequence_length):
            window = self.scaled_data[i : i + sequence_length, : ]
            self.images.append(window)
            self.output.append(self.scaled_data[i + sequence_length, 3])

    def __getitem__(self, idx):
        x = self.images[idx]
        y = self.output[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.images)

In [4]:
sequence_length = 15
batch_size = 16

dataset_instance = dataset(data.values, sequence_length = sequence_length)
trainloader = DataLoader(dataset=dataset_instance, batch_size = batch_size, drop_last = True)

for idx, (x,y) in enumerate(trainloader):
    print(x.shape)
    if idx==0:
        break

torch.Size([16, 15, 5])


In [5]:
class cnn_for_time_series(nn.Module):
    def __init__(self, hidden_size, out_size, dropout_prob = 0.2):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = hidden_size, kernel_size = (3,3), stride = 1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size = (2,2), stride = 1)
        self.drop = nn.Dropout(p=dropout_prob)

        self.conv2 = nn.Conv2d(in_channels = hidden_size, out_channels = out_size, kernel_size = (3,3), stride = 1)

        self.fc1 = nn.Linear(out_size * 12 * 2, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):

        x = self.conv1(x)
        x = self.pool(x)
        x = self.drop(x)

        x = self.conv2(x)
        x = self.drop(x)

        x = torch.flatten(x,1)

        x = self.fc1(x)
        x = self.fc2(x)

        return x

In [6]:
hidden_size1 = 128
hidden_size2 = 64

model = cnn_for_time_series(hidden_size1, hidden_size2, dropout_prob=0.3)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.002)
criterion = nn.MSELoss()

In [7]:
num_epochs = 50

scaler = MinMaxScaler(feature_range=(0,1))

for epoch in range (num_epochs):

    epoch_loss = 0

    for  x, y in trainloader:
        optimizer.zero_grad()

        x = x.unsqueeze(1)
        y_pred = model(x)

        loss = criterion(y_pred, y.view(-1,1))
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()
    print("Loss at epoch ",epoch," is : ", epoch_loss/len(trainloader))


Loss at epoch  0  is :  0.06863468119020884
Loss at epoch  1  is :  0.01976430570357479
Loss at epoch  2  is :  0.015805691548545536
Loss at epoch  3  is :  0.014804621780058369
Loss at epoch  4  is :  0.015698538860306144
Loss at epoch  5  is :  0.0141152617240247
Loss at epoch  6  is :  0.014610596782101007
Loss at epoch  7  is :  0.013246170686519084
Loss at epoch  8  is :  0.013778516877209768
Loss at epoch  9  is :  0.014360758514764408
Loss at epoch  10  is :  0.013985882920678705
Loss at epoch  11  is :  0.012537618516944349
Loss at epoch  12  is :  0.01325042086925047
Loss at epoch  13  is :  0.012709540111245587
Loss at epoch  14  is :  0.011392865242669359
Loss at epoch  15  is :  0.012046171912516002
Loss at epoch  16  is :  0.011703497109313806
Loss at epoch  17  is :  0.010052960909282168
Loss at epoch  18  is :  0.012335426872596145
Loss at epoch  19  is :  0.01129899084723244
Loss at epoch  20  is :  0.010691329720430076
Loss at epoch  21  is :  0.011016819616391635
Loss

In [8]:
model.eval()
with torch.no_grad():
    test_input = x[5]  # Use the last batch as a test input
    test_output = y[5]
    print('Actual Close : ',test_output)
    test_input = test_input.unsqueeze(0)
    prediction = model(test_input)
    predicted_close = prediction.numpy()  # Get the predicted 'Close' price
    print(f"Predicted Close: {predicted_close}")

Actual Close :  tensor(0.4337)
Predicted Close: [[0.33372575]]


In [9]:
data = yf.download(stock_symbol, period="5d", interval="30m")

scaler = MinMaxScaler(feature_range = (0,1))

# Get the last 15 data points
last_15_data = data.tail(15)

pred_input = last_15_data.loc[:,['Open', 'High', 'Low', 'Close', 'Volume']]
scaled_input = scaler.fit_transform(pred_input)



pred_tensor = torch.tensor(scaled_input , dtype=torch.float32)

with torch.no_grad():
    pred_tensor = pred_tensor.unsqueeze(0)
    pred_tensor = pred_tensor.unsqueeze(1)
    print(pred_tensor.shape)
    pred_output = model(pred_tensor)
    print("shape : ", pred_output.shape)

    # predicted_output = pred_output.numpy()
    
    # scaler.inverse_transform(predicted_output.reshape(-1,1))
    scaled_pred = pred_output.detach().numpy()[0, 0]
    
    # Create a dummy array with zeros for other features
    dummy_array = np.zeros((1, 5))  # 5 features: Open, High, Low, Close, Volume
    dummy_array[0, 3] = scaled_pred  # Put prediction in Close price position (index 3)
    
    # Now inverse transform
    predicted_value = scaler.inverse_transform(dummy_array)[0, 3]  # Get only the Close price

    print(f"Predicted Close Price: ₹{predicted_value:.2f}")
    
    # Optional: Compare with actual
    actual_close = float(data['Close'].iloc[-1])
    print(f"Actual Close Price: ₹{actual_close:.2f}")
    print(f"Prediction Error: ₹{abs(actual_close - predicted_value):.2f}")

[*********************100%***********************]  1 of 1 completed

torch.Size([1, 1, 15, 5])
shape :  torch.Size([1, 1])
Predicted Close Price: ₹1261.88
Actual Close Price: ₹1264.60
Prediction Error: ₹2.72



C:\Users\GIDEON\AppData\Local\Temp\ipykernel_34964\3611095557.py:37: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  actual_close = float(data['Close'].iloc[-1])


In [10]:
import joblib

# Save the model
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': num_epochs,
    'loss': epoch_loss,
}, 'stock_prediction_model.pth')

# Save the scaler
joblib.dump(scaler, 'stock_scaler.pkl')

['stock_scaler.pkl']

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\GIDEON\anaconda\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\GIDEON\anaconda\Lib\asyncio\selector_events.py", line 132, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\GIDEON\anaconda\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\GIDEON\anaconda\Lib\asyncio\selector_events.py", line 132, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionReset